In [21]:
!pip install textblob

   ---------------------------------------- 0.0/624.3 kB ? eta -:--:--
   ---------------------------------------- 0.0/624.3 kB ? eta -:--:--
   ---------------------------------------- 624.3/624.3 kB 9.8 MB/s  0:00:00


In [32]:
!pip install tensorflow

In [50]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

df = pd.read_csv("brooklyn99 script.csv")
lines = df["line"].astype(str).tolist()

tokenizer = Tokenizer(oov_token="<OOV>")
tokenizer.fit_on_texts(lines)
total_words = len(tokenizer.word_index) + 1

input_sequences = []

for line in lines:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        input_sequences.append(token_list[:i + 1])

max_sequence_len = max(len(x) for x in input_sequences)
input_sequences = pad_sequences(input_sequences, maxlen=max_sequence_len, padding="pre")

X = input_sequences[:, :-1]
y = input_sequences[:, -1]
y = tf.keras.utils.to_categorical(y, num_classes=total_words)

model = tf.keras.Sequential(
[
    tf.keras.layers.Embedding(total_words, 128, input_length=max_sequence_len - 1),
    tf.keras.layers.LSTM(100),
    tf.keras.layers.Dense(total_words, activation="softmax")
]
)

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

model.fit(X, y, epochs=25, batch_size=64, verbose=1)


C:\Users\jashv\anaconda3\Lib\site-packages\keras\src\layers\core\embedding.py:100: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/25
1111/1111 ━━━━━━━━━━━━━━━━━━━━ 150s 129ms/step - accuracy: 0.0365 - loss: 6.7118
Epoch 2/25
1111/1111 ━━━━━━━━━━━━━━━━━━━━ 133s 119ms/step - accuracy: 0.0632 - loss: 6.1747
Epoch 3/25
1111/1111 ━━━━━━━━━━━━━━━━━━━━ 159s 135ms/step - accuracy: 0.0915 - loss: 5.8189
Epoch 4/25
1111/1111 ━━━━━━━━━━━━━━━━━━━━ 144s 130ms/step - accuracy: 0.1135 - loss: 5.5479
Epoch 5/25
1111/1111 ━━━━━━━━━━━━━━━━━━━━ 132s 119ms/step - accuracy: 0.1280 - loss: 5.3218
Epoch 6/25
1111/1111 ━━━━━━━━━━━━━━━━━━━━ 142s 128ms/step - accuracy: 0.1404 - loss: 5.1208
Epoch 7/25
1111/1111 ━━━━━━━━━━━━━━━━━━━━ 126s 113ms/step - accuracy: 0.1512 - loss: 4.9337
Epoch 8/25
1111/1111 ━━━━━━━━━━━━━━━━━━━━ 127s 114ms/step - accuracy: 0.1622 - loss: 4.7546
Epoch 9/25
1111/1111 ━━━━━━━━━━━━━━━━━━━━ 126s 113ms/step - accuracy: 0.1739 - loss: 4.5800
Epoch 10/25
1111/1111 ━━━━━━━━━━━━━━━━━━━━ 150s 135ms/step - accuracy: 0.1887 - loss: 4.4096
Epoch 11/25
1111/1111 ━━━━━━━━━━━━━━━━━━━━ 130s 117ms/step - accuracy: 0.2047 -

In [56]:
def generate_text(seed_text, next_words, temperature=0.7):
    banned = ["sex", "love", "kiss", "bed", "naked", "butt", "dumped", "hook", "sleep"]

    words = seed_text.split()

    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([" ".join(words)])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len - 1, padding="pre")

        preds = model.predict(token_list, verbose=0)[0]

        for w in banned:
            if w in tokenizer.word_index:
                preds[tokenizer.word_index[w]] *= 0.01

        preds = np.log(preds + 1e-9) / temperature
        preds = np.exp(preds) / np.sum(np.exp(preds))

        predicted = np.random.choice(len(preds), p=preds)

        for word, index in tokenizer.word_index.items():
            if index == predicted:
                words.append(word)
                if word in [".", "!", "?"]:
                    return " ".join(words)
                break

        if len(words) >= 16:
            return " ".join(words)

    return " ".join(words)


In [57]:
print(generate_text("jake peralta", 16, temperature=0.85))
print(generate_text("captain holt", 12, temperature=0.5))

jake peralta just picked up a family with a dog paulie i think it's a sudden
captain holt are you trying to know what i'm so proud of you guys
